In [4]:
import pandas as pd
from CRPS import CRPS
from sklearn.metrics import mean_pinball_loss

In [7]:
df_lasso = pd.read_csv("../../results/predictions_lasso_1.csv")
df_scad = pd.read_csv("../../results/predictions_scad_1.csv")

In [14]:
# import data
hour = 17
data = pd.read_csv(f"../../data/final_{hour}.csv")
data["datetime"] = pd.to_datetime(data["datetime"])

In [15]:
# make lagged variables
data["priceBE_lag1"] = data["priceBE"].shift(1)
data["priceBE_lag2"] = data["priceBE"].shift(2)
data["priceBE_lag3"] = data["priceBE"].shift(3)
data["priceBE_lag4"] = data["priceBE"].shift(4)
data["priceBE_lag5"] = data["priceBE"].shift(5)

In [11]:
df_lasso.iloc[:,0]

0        58.783028
1        58.792971
2        58.767991
3        58.770671
4        58.866332
           ...    
1781    162.802394
1782    162.882086
1783    162.731264
1784    162.816050
1785    164.745764
Name: X1, Length: 1786, dtype: float64

In [ ]:
for i in ran
mean_pinball_loss(y_test, predictions["0.05"], alpha=0.05)

In [13]:
for i in range(0,9):
    mean_pinball_loss(y_test, df_lasso.iloc[:,i], alpha=(i+1)/10)

0
1
2
3
4
5
6
7
8
